In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [83]:
#Reading datafile
data = pd.read_csv("data.csv", header = None)
data = data.drop(index=0) #Drops the column names
#print(data.shape) #dimensions of the data
#print(data)

#Features and label extraction
data_x = data.iloc[:, :48] #Features
data_y = data.iloc[:, 48] #Labels
from sklearn import preprocessing
MinMaxScaler = preprocessing.MinMaxScaler()
data_x = MinMaxScaler.fit_transform(data_x)

#Splittiing data into train (80%) and test (20%)
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size = 0.2)
print("train_y in main")
print(train_y.shape)
#print(data.columns.values)


train_y in main
(32764,)


In [84]:
def Accuracy(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float    
    """
    cm = ConfusionMatrix(y_true,y_pred)
    correct = np.trace(cm)
    total = cm.sum()
    acc=0
    if(total>0):
        acc = correct/total
    return acc

In [85]:
def ConfusionMatrix(y_true,y_pred):
    """
    :type y_true: numpy.ndarray
    :type y_pred: numpy.ndarray
    :rtype: float
    """
    print("y_true")
    print(y_true.shape)
    print("y_pred")
    print(y_pred.shape)
    #For 0-10 range
    un = np.unique(y_true)
    y_true=y_true-un[0]
    un = np.unique(y_pred)
    y_pred=y_pred-un[0]
    classes = len(np.unique(np.concatenate((y_true,y_pred))))        
    mul = y_true*classes
    sum1 = mul+y_pred    
    x,bins= np.histogram(sum1,bins=np.arange(classes**2+1))
    mat = x.reshape(classes, classes)
    return(mat)

In [86]:
def PCA(X_train,N):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: numpy.ndarray
    """
    #standardizing data 
    sc = StandardScaler()
    std_data = sc.fit_transform(X_train)
    pca_data = pd.DataFrame(std_data)

    #Construct Covariance Matrix(C)
    transpose_data = pca_data.T
    C = np.cov(transpose_data)


    #Decomposing C to eigen vectors and values
    e_val, e_vec =  np.linalg.eig(C)


    #N = int(input("Reduced Deminsions:")) ------ Passed on to tge function

    #Estimating High-valued Eigen vectors
    eval_sorted = np.argsort(e_val)
    top_evals = eval_sorted[::-1][:N]
    high_valued_vectors = e_vec[:,top_evals]

    PCA = np.empty([pca_data.shape[0], high_valued_vectors.shape[1]])
    i = 0
    for v in high_valued_vectors.T:
            PCA[:,i] = np.dot(pca_data, v.T)
            i += 1
    return PCA

In [102]:
def gini(samples, list_ind):
    n_samples = len(samples)
    if n_samples != 0:
        class_ind = [dp[-1] for dp in samples]
        gini_index = 0.0
        for index in list_ind:
            prob = class_ind.count(index) / n_samples
            gini_index += prob ** 2
        return 1.0 - gini_index
    return 0

def entropy(samples, index_list):
    n_samples = len(samples)
    if n_samples != 0:
        labels = [dp[-1] for dp in samples]
        sum_ = 0.0
        for index in index_list:
            prob = labels.count(index) / n_samples
            sum_ -= prob * np.log2(prob)
        return sum_
    return 0

def gini_gain(samples, list_ind):
    n_samples = sum([len(dp) for dp in samples])
    data = [sample for dp in samples for sample in dp]
    gini_impurity = gini(data, list_ind) #Gini index of parent node
    for dp in samples:
        gini_impurity -= gini(dp, list_ind) * len(dp) / n_samples #Gini index of child node
    return gini_impurity

def feature_split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        cls_list = [row[-1] for row in (left + right)]
        node['left'] = node['right'] = max(set(cls_list), key = cls_list.count)
        return
    if depth >= max_depth:
        left_list = [row[-1] for row in left]
        node['left'] = max(set(left_list), key = left_list.count)
        right_list = [row[-1] for row in right]
        node['right'] = max(set(right_list), key = right_list.count)
        return
    if len(left) <= min_size:
        left_list = [row[-1] for row in left]
        node['left'] = max(set(left_list), key = left_list.count)
    else:
        feature_index = int( np.random.random() * (len(right[0]) - 1) )
        node['left'] = optimal_split(left, feature_index)
        feature_split(node['left'], max_depth, min_size, depth + 1)

    if len(right) <= min_size:
        right_list = [row[-1] for row in right]
        node['right'] = max(set(right_list), key = right_list.count)
    else:
        feature_index = int( np.random.random() * (len(right[0]) - 1) )
        node['right'] = optimal_split(right, feature_index)
        feature_split(node['right'], max_depth, min_size, depth + 1)

def optimal_split(samples, index):
    max_gain = -1.0
    list_ind = list(set(dp[-1] for dp in samples))
    value = 0.0
    split_groups = None
    for dp in samples:
        left = []
        right = []
        for sample in samples:
            if sample[index] < dp[index]:
                left.append(sample)
            else:
                right.append(sample)
        sample_set = [left, right]
        info_gain = gini_gain(sample_set, list_ind)
        if info_gain > max_gain:
            value, max_gain, split_groups = dp[index], info_gain, sample_set
    return { 'index': index, 'split_value': value, 'groups': sample_set }

def DecisionTree(samples, max_depth, min_size):
    max_gain = -1.0
    list_ind = list(set(row[-1] for row in samples))
    value = 0.0
    split_groups = None
    feature_ind = int(np.random.random() * (len(samples[0]) - 1) )
    for row in samples:
        left = []
        right = []
        for sample in samples:
            if sample[feature_ind] < row[feature_ind]:
                left.append(sample)
            else:
                right.append(sample)
        data = [left, right]
        info_gain = gini_gain(data, list_ind)
        if info_gain > max_gain:
            value, max_gain, split_groups = row[feature_ind], info_gain, data 
    tree = {'index': feature_ind,'split_value': value,'groups': data }
    feature_split(tree, max_depth, min_size, 1)
    return tree

def tree_nodes(feature, samples):
    if samples[feature['index']] < feature['split_value']:
        if isinstance(feature['left'], dict):
            return tree_nodes(feature['left'], samples)
        else:
            return feature['left']
    elif isinstance(feature['right'], dict):
        return tree_nodes(feature['right'], samples)
    else:
        return feature['right']
    
def RandomForest(X_train,Y_train,X_test,Y_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: numpy.ndarray
    """
    labels = []
    trees = []
    n_trees = 200
    n_features = X_train.shape[1]
    max_depth = 10
    min_samples = 5
    data = np.concatenate((X_train, Y_train), axis = 1)

    for i in range(n_trees):
        #Random selection of features for decision trees
        feature_ind = np.random.choice(len(data), n_features)
        trees.append(DecisionTree(data[feature_ind], max_depth, min_samples))
        
    for sample in X_train:
        classes = []
        for tree in trees:
            classes.append(tree_nodes(tree, sample))
        labels.append(int(max(set(classes), key = classes.count)))    
    return labels

pred = RandomForest(train_x, train_y, test_x, test_y)
pred = np.array(pred)
pred = pred.reshape(pred.shape[0], 1)
rand_pred = np.random.randint(1,11, size=(32764, 1))
rand_pred = np.ones((32764,1),dtype=int)
rand_pred = np.full((32764, 1), 2)
print(rand_pred)
print("Random Forest accuracy : %f" %Accuracy(train_y, pred))
print("random Random Forest accuracy : %f" %Accuracy(train_y, rand_pred))


[[2]
 [2]
 [2]
 ...
 [2]
 [2]
 [2]]
y_true
(32764, 1)
y_pred
(32764, 1)
Random Forest accuracy : 0.257020
y_true
(32764, 1)
y_pred
(32764, 1)
random Random Forest accuracy : 0.090313


In [87]:
def gini(group, list_of_class_ids):
    Ngroup = len(group)
    #print("Ngroup")
    #print(Ngroup)
    if Ngroup == 0:
        return 0
    dataset_class_ids = [row[-1] for row in group]
    sum_over_classes = 0.
    for class_id in list_of_class_ids:
        prob = dataset_class_ids.count(class_id)/Ngroup
        sum_over_classes += prob**2
    return 1. - sum_over_classes

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'] = to_terminal(left)
        node['right'] = to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        feature_index = int( np.random.random()*(len(right[0]) - 1) )
        node['left'] = get_split(left, feature_index)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        feature_index = int( np.random.random()*(len(right[0]) - 1) )
        node['right'] = get_split(right, feature_index)
        split(node['right'], max_depth, min_size, depth+1)
        
def calc_information_gain(groups, list_of_class_ids):
    # count all samples
    Nall = sum([len(group) for group in groups])
    
    # calculate Gini index of parent node
    all_rows = [row for group in groups for row in group]
    IG = gini(all_rows, list_of_class_ids)
    
    # calculate Gini index of daughter nodes
    for group in groups:
        IG -= gini(group, list_of_class_ids)*len(group)/Nall
    return IG

def split_node(index, value, dataset):
    ''' Split the dataset into two using a feature index and 
    feature value '''
    left = []
    right = []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return [left, right]

def get_split(dataset, index):
    ''' Evaluate all possible splits given the dataset and the index of 
    the feature on which to split '''
    list_of_class_ids = list(set(row[-1] for row in dataset))
    split_value, max_IG, split_groups = 0., -1., None
    for row in dataset:
        groups = split_node(index, row[index], dataset)
        IG = calc_information_gain(groups, list_of_class_ids)
        if IG > max_IG:
            split_value, max_IG, split_groups = row[index], IG, groups
    return {'index': index,'split_value': split_value,'groups': groups }

def build_tree(train, max_depth, min_size):
    # randomly determine the feature index
    feature_index = int( np.random.random()*(len(train[0]) - 1) )
    #print("initial feature index")
    #print(feature_index)
    #print("PCA")
    trainb = np.delete(train, -1, axis=1)
    ret = PCA(trainb,1)
    #print(ret.shape)
    #print(ret)
    ret = ret.flatten()
    #print(ret.shape)
    feature_index = np.argmax(ret)
    #print("new feature index")
    #print(feature_index)
    
    root = get_split(train, feature_index)
    split(root, max_depth, min_size, 1)
    return root

def to_terminal(group):
    # Create a terminal node value
    list_of_classes = [row[-1] for row in group]
    return max(set(list_of_classes), key=list_of_classes.count)

def traverse_tree(node, row):
    if row[node['index']] < node['split_value']:
        if isinstance(node['left'], dict):
            return traverse_tree(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return traverse_tree(node['right'], row)
        else:
            return node['right']

def RandomForest(X_train,Y_train,X_test,Y_test):
    """
    :type X_train: numpy.ndarray
    :type X_test: numpy.ndarray
    :type Y_train: numpy.ndarray
    
    :rtype: numpy.ndarray
    """
    labels = []
    n_trees = 200
    n_features = X_train.shape[1]
    max_depth = 35
    min_samples = 5
    
    print("train_X")
    print(X_train.shape)
    
    print("Y_train")
    print(Y_train.shape)
    
    data = np.concatenate((X_train, Y_train), axis = 1)
    c, cot = np.unique(Y_train, return_counts = True)
    print("Unique for Y-train")
    print(c)
    print(cot)
    print(np.argmax(cot))
    
    forest = []
    for i in range(n_trees):
        feature_ind = np.random.choice(len(data), n_features)
        forest.append(build_tree(data[feature_ind], max_depth, min_samples))
        
    for sample in X_train:
        classes = []
        for root in forest:
            classes.append(traverse_tree(root, sample))
        labels.append(int(max(set(classes), key = classes.count)))
        
    return labels

print("trin_y")
print(train_y.shape)
train_y = train_y.values
train_y = train_y.reshape(train_y.shape[0],1)
print("train_y")
print(train_y.shape)

pred = RandomForest(train_x, train_y, test_x, test_y)
pred = np.array(pred)
pred = pred.reshape(pred.shape[0],1)
c, cot = np.unique(pred, return_counts = True)
print("Unique")
print(c)
print(cot)
print("Random Forest accuracy : %f" %Accuracy(train_y, pred))

trin_y
(32764,)
train_y
(32764, 1)
train_X
(32764, 48)
Y_train
(32764, 1)
Unique for Y-train
[ 1  2  3  4  5  6  7  8  9 10 11]
[2959 2998 3020 2968 2977 2986 2952 2969 2955 2997 2983]
2
Unique
[ 1  2  3  6  7  8  9 10 11]
[ 3206  2413   202     1   115 23007   758    13  3049]
y_true
(32764, 1)
y_pred
(32764, 1)
Random Forest accuracy : 0.232053
